# BDV - Recipe analysis

## Imports

In [3]:
from pyspark.sql.functions import round, avg, count, mean, explode, split, col
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('recipe-analysis') \
    .getOrCreate()
    
spark

## Transform and Load Data

In [4]:
recipes = spark.read.option("delimiter", ";").csv("./data/clean_recipes.csv", header=True, inferSchema=True)
reviews = spark.read.option("delimiter", ",").csv("./data/clean_reviews.csv", header=True, inferSchema=True)

In [5]:
recipes.limit(5).toPandas()

,Recipe Name,Review Count,Recipe Photo,Author,Prepare Time,Cook Time,Total Time,Ingredients,Directions,RecipeID
0,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,25 m,15 m,3 h 10 m,"yeast,water,white sugar,salt,egg,butter,flour,...","Dissolve yeast in warm water.**Stir in sugar, ...",7000
1,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,15 m,1 h,1 h 20 m,"flour,salt,baking powder,poppy,butter,vegetabl...",'Preheat oven to 350 degrees F (175 degrees C)...,7001
2,Applesauce Bread I Recipe,124,https://images.media-allrecipes.com/userphotos...,GAF55,10 m,1 h 20 m,1 h 30 m,"flour,egg,white sugar,vegetable oil,applesauce...",Preheat oven to 350 degrees F (175 degrees C)....,7003
3,Apple Raisin Bread Recipe,39,https://images.media-allrecipes.com/userphotos...,Helen Hanson,15 m,1 h,1 h 15 m,"flour,baking powder,baking soda,salt,cinnamon,...",Preheat oven to 350 degrees F (175 degrees C)....,7006
4,Buttermilk Oatmeal Bread Recipe,41,https://images.media-allrecipes.com/userphotos...,Helen Hanson,10 m,1 h,1 h 40 m,"oat,buttermilk,vegetable oil,egg,brown sugar,f...",Mix oats with buttermilk. Let stand for 1/2 h...,7007


In [6]:
reviews.limit(5).toPandas()

,RecipeID,profileID,Rate
0,7000,675719,5.0
1,7000,1478626,5.0
2,7000,608663,5.0
3,7000,2785736,5.0
4,7000,594474,5.0


## Hard Facts

In [7]:
ingredients = recipes.select(explode(split(col("Ingredients"), ",")).alias("Ingredient"), "RecipeID")
best_recipes = reviews\
    .groupBy("RecipeID")\
    .agg(mean("Rate"), count("Rate"))\
    .withColumnRenamed("avg(Rate)", "AvgRate")\
    .withColumnRenamed("count(Rate)", "CountRate")\
    .orderBy("avg(Rate)", ascending=[0])

print(f"recipes.count(): {recipes.count()}")
print(f"reviews.count(): {reviews.count()}")
print(f"ingredients.count(): {ingredients.count()}")
print(f"best_recipes.limit(10):"); best_recipes.limit(10).toPandas()

recipes.count(): 12351
reviews.count(): 1563566
ingredients.count(): 102517
best_recipes.limit(10):


,RecipeID,AvgRate,CountRate
0,20441,5.000000,30
1,9263,5.000000,30
2,24311,5.000000,1
3,13603,4.966667,30
4,10819,4.965517,29
5,15297,4.964286,28
6,19207,4.933333,30
7,13481,4.933333,30
8,24221,4.933333,30
9,9153,4.933333,30


## Research Questions
### 1. Welche Zutat wird am häufigsten verwendet?

In [8]:
ingredients.limit(5).toPandas()

,Ingredient,RecipeID
0,yeast,7000
1,water,7000
2,white sugar,7000
3,salt,7000
4,egg,7000


In [ ]:
ingredients\
    .groupBy("Ingredient")\
    .count()\
    .orderBy("count", ascending=[0])\
    .withColumnRenamed("count", "Count")\
    .limit(10)\
    .toPandas()

### 2. Welche Zutat wird am häufigsten in den 10 bestbewerteten Rezepten verwendet?

In [ ]:
best_ten_recipes = best_recipes\
    .join(recipes, on="RecipeID", how="inner")\
    .orderBy("AvgRate", ascending=[0])\
    .limit(10)

best_ten_recipes.toPandas()

In [ ]:
best_ten_recipes\
    .join(ingredients, on="RecipeID", how="inner")\
    .groupBy("Ingredient")\
    .count()\
    .orderBy("count", ascending=[0])\
    .withColumnRenamed("count", "Count")\
    .limit(5)\
    .toPandas()

### 3. Wie lange ist die durchschnittliche Zubereitungszeit für ein Rezept?

### 4. Wie gut sind die Rezepte mit der längsten Zubereitungszeit bewertet?

### 5. Können wir anhand der Zutaten, der Zubereitungsschritte und der Zubereitungszeit vorhersagen wie gut die Bewertung ausfallen wird?